In [313]:
import sys
import os

# Workaround to make packages work in both Jupyter notebook and Python
MODULE_ROOT_NAME = "AgeEstimator"
MODULE_PATHS = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
MODULE_PATHS = list(
    filter(lambda x: x.endswith(MODULE_ROOT_NAME), MODULE_PATHS))
MODULE_PATH = MODULE_PATHS[0] if len(MODULE_PATHS) == 1 else ""
if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)
    
from server.data.dataset import DataLoader
from server.models.cnn.model import get_model, OLD_WEIGHTS_PATH, BEST_WEIGHTS_PATH, LABEL_MAPPING, get_models, N_CLASSES, IMAGE_SIZE

In [314]:
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.image as img
import numpy as np
import pandas
import multiprocessing

## Global variables

In [315]:
batch_size = 64

## Pocessing

In [316]:
def get_class_to_age_map():
    return pandas.read_csv("./class_to_estimated_age.csv", dtype=float).Age.to_dict()

In [317]:
def get_label_to_category_map():
    unique_labels = list(set(LABEL_MAPPING.values()))
    category_map = {class_label: inx for inx, class_label in enumerate(unique_labels)}
    category_map_r = {inx: class_label for inx, class_label in enumerate(unique_labels)}
    return category_map, category_map_r

In [318]:
def normalize_label(y):
    category_map, _ = get_label_to_category_map()
    normalize = lambda x:category_map[LABEL_MAPPING[x]]
    labels = np.vectorize(normalize)(y)
    return to_categorical(labels, N_CLASSES)

In [319]:
def get_img_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    
    valid_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    return train_datagen, valid_datagen, test_datagen

In [320]:
def to_generator(datagen, dataframe, directory, batch_size=batch_size):
    g = datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col="FilePath",
        y_col="Age",
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
#         class_mode='sparse',
        class_mode="categorical"
    )

    # Convert to tf.data to better utilize multiprocessing
    n_class = len(np.unique(np.array(dataframe["Age"])))
    tf_g = tf.data.Dataset.from_generator(lambda: g,
        output_types=(tf.float32, tf.float32),
        output_shapes=(
            tf.TensorShape([None, IMAGE_SIZE[0], IMAGE_SIZE[1], 3]), 
            tf.TensorShape([None, 55])
        )
    )

    return tf_g

## Metrics

In [321]:
class_2_age = get_class_to_age_map()

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(class_2_age.keys()),
        values=list(class_2_age.values()),
        key_dtype=tf.int32, value_dtype=tf.float32
    ),
    default_value=tf.constant(-1.0),
    name="class_weight"
)

age_tensor = K.map_fn(lambda x: table.lookup(x),
                      K.arange(len(class_2_age)),
                      dtype=tf.float32)

def mae_all_class(y_true, y_pred):
    r"""Mean absolute error of the true class label and the average prediction."""
    if not tf.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
        
    # Calculate average prediction
    age_pred = tf.tensordot(y_pred, age_tensor, axes=1)
    
    # Calculate true age
    y_true = K.cast(y_true, y_pred.dtype)
    y_true_idx = tf.math.argmax(y_true, axis=1, output_type=tf.int32)
    age_true = table.lookup(y_true_idx)
    return tf.math.reduce_mean(tf.math.abs(age_pred - age_true))

## Sampling and train/valid Split

In [322]:
def get_dataframe(x, y, name, sample_size=0):
    # Stack to [[img, label], ...] matrix
    stk = np.column_stack((x, y))
    
    # Save as csv
    np.savetxt("%s.csv" % (name), stk, fmt="%s", delimiter=",", comments="", header="FilePath,Age")
    
    # `flow_from_dataframe` requires loading labels as string
    df = pandas.read_csv("./%s.csv" % (name), dtype=str)
    
    return df if sample_size == 0 else df.sample(n=sample_size)

In [323]:
def split_train_valid(df):
    train_df = df.sample(frac=0.9)
    validation_df = df.drop(train_df.index)
    return train_df, validation_df

## Training Utilities

In [324]:
def get_callbacks(log_dir):
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

    # Don't waste our time/resource on bad training
    es = EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1,
        patience=100)
    
    tb = TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_grads=False,
        write_images=False,
        embeddings_freq=0,
        embeddings_layer_names=None,
        embeddings_metadata=None,
        embeddings_data=None,
        update_freq='epoch')
    
    # Save the best weight seen so far
    mc = ModelCheckpoint(
        BEST_WEIGHTS_PATH,
#         monitor='val_loss',
#         mode='min',
        monitor='val_categorical_accuracy',
        mode='max',
        verbose=1,
        save_weights_only=True,
        save_best_only=True)
    
    # Modify the best score for retrains
    mc.best = 0.14363
    
    # Try to get rid of local minimum
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=20,
        min_lr=0.000001)
    
    return [mc, es, tb, reduce_lr]

In [325]:
def get_log_dir():
    log_i = 0
    log_dir = "logs/run_"
    
    while os.path.exists(log_dir + str(log_i)):
        log_i += 1

    return log_dir + str(log_i)

In [326]:
def compare_results(y_true, y_predict, top_n=5):
    r"""Compare the last 10 result of top 5 prediction and its label."""
    y_hat = y_predict.argsort(axis=1)[:,-top_n:]
    y_true = np.argmax(y_true, axis=1)
    print(y_hat[-10:])
    print(y_true[-10:])

## Training

### Train a lot of models

Train with a small portion of our dataset to compare the performace of the combinations of hyperparameters, so we can decide which model should be trained with a larger epochs.

In [330]:
def train_many(train_generator, valid_generator, train_len, valid_len):
    epochs = 20
    models = get_models()
    
    for m in models:
        model_name, optimizer, model = m
        print("== Training %s ==" % model_name)

        model.compile(loss="categorical_crossentropy", optimizer=optimizer, \
                      metrics=[mae_all_class ,"categorical_accuracy"])

        log_dir = get_log_dir()
        callbacks = get_callbacks(log_dir + "/%s" % model_name)

        model.fit(
            x=train_generator,
            steps_per_epoch=train_len // batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=valid_generator,
            validation_steps=valid_len // batch_size,
            callbacks=callbacks,
            workers=max(2, multiprocessing.cpu_count() - 2),
            use_multiprocessing=True
        )

        model.save_weights("%s_weight.hdf5" % model_name)

    return model

### Train the finalized model

In [331]:
def train(x, y):
    epochs = 1000
    
    optimizer = Nadam(lr=0.00007, beta_1=0.9, beta_2=0.999)

    model = get_model()
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, \
        metrics=[mae_all_class, "categorical_accuracy"])
    
    if os.path.exists(BEST_WEIGHTS_PATH):
        model.load_weights(BEST_WEIGHTS_PATH)
        print("best weight [%s] loaded." % BEST_WEIGHTS_PATH)
#     elif os.path.exists(OLD_WEIGHTS_PATH):
#         model.load_weights(OLD_WEIGHTS_PATH)
#         print("old weight [%s] loaded." % OLD_WEIGHTS_PATH)
    else:
        print("fresh start.")
            
    log_dir = get_log_dir()
    callbacks = get_callbacks(log_dir)

    train_len = round(len(x) * 0.9)
    
    model.fit(
        x=x,
        y=y,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_split=0.1,
        shuffle=True,
        callbacks=callbacks,
        workers=max(2, multiprocessing.cpu_count() - 2),
        use_multiprocessing=True
    )
    
    model.save_weights(OLD_WEIGHTS_PATH)

    return model

### Entry point

In [332]:
def main(sample_size=0, is_final_model=True):
    dl = DataLoader()
    use_bottleneck_features = True
    x_train, y_train = dl.load_train(use_bottleneck_features)
    x_test, y_test = dl.load_test(use_bottleneck_features)
    
    # Discretizate the continuous age into ordinal labels and map it with one-hot encoding
    y_train = normalize_label(y_train)
    y_test = normalize_label(y_test)
    
#     # The size is too large, so build a csv file for (image_filename/label) mapping
#     train_df = get_dataframe(x_train, y_train, "train", sample_size=sample_size)
#     train_df, valid_df = split_train_valid(train_df)
#     test_df = get_dataframe(x_test, y_test, "test", sample_size=sample_size // 10)

#     # Data augmentation for training set
#     train_datagen, valid_datagen, test_datagen = get_img_generators()
#     train_generator = to_generator(train_datagen, train_df, dl.train_dir)
#     valid_generator = to_generator(valid_datagen, valid_df, dl.train_dir)
#     test_generator = to_generator(test_datagen, test_df, dl.test_dir)
    
#     train_len = len(x_train)
#     valid_len = len(valid_df)
    test_len = len(x_test)
    
    if is_final_model:
        # If it's a finalized model, train with a larger epochs
        trained_model = train(x_train, y_train)

        evaluation = trained_model.evaluate(
            x=x_test, y=y_test)
        y_hat = trained_model.predict(
            x=x_test)
        
        print(evaluation)
        compare_results(y_test, y_hat)

        return evaluation, y_hat, y_test
    
#     else:
#         train_many(train_generator, valid_generator, train_len, valid_len)

In [333]:
res = main()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2048)]            0         
_________________________________________________________________
d0 (Dense)                   (None, 1024)              2098176   
_________________________________________________________________
bn0 (BatchNormalization)     (None, 1024)              4096      
_________________________________________________________________
d1 (Dense)                   (None, 512)               524800    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 512)               2048      
_________________________________________________________________
d2 (Dense)                   (None, 256)               131328    
_________________________________________________________________
bn2 (BatchNormalization)     (None, 256)               1024  

Epoch 17/1000
134080/134466 [============================>.] - ETA: 0s - loss: 1.2999 - mae_all_class: 1.4213 - categorical_accuracy: 0.5386
Epoch 00017: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.3001 - mae_all_class: 1.4263 - categorical_accuracy: 0.5385 - val_loss: 4.6844 - val_mae_all_class: 4.3271 - val_categorical_accuracy: 0.1413
Epoch 18/1000
134016/134466 [============================>.] - ETA: 0s - loss: 1.2975 - mae_all_class: 1.4069 - categorical_accuracy: 0.5398
Epoch 00018: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.2974 - mae_all_class: 1.4066 - categorical_accuracy: 0.5399 - val_loss: 4.6819 - val_mae_all_class: 4.3357 - val_categorical_accuracy: 0.1410
Epoch 19/1000
134016/134466 [============================>.] - ETA: 0s - loss: 1.2999 - mae_all_class: 1.4231 - categorical_accuracy: 0.5413
Epoch 0

134016/134466 [============================>.] - ETA: 0s - loss: 1.2779 - mae_all_class: 1.3939 - categorical_accuracy: 0.5466
Epoch 00036: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.2781 - mae_all_class: 1.4025 - categorical_accuracy: 0.5466 - val_loss: 4.7449 - val_mae_all_class: 4.3408 - val_categorical_accuracy: 0.1412
Epoch 37/1000
134208/134466 [============================>.] - ETA: 0s - loss: 1.2774 - mae_all_class: 1.4033 - categorical_accuracy: 0.5471
Epoch 00037: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 107us/sample - loss: 1.2777 - mae_all_class: 1.4074 - categorical_accuracy: 0.5471 - val_loss: 4.7323 - val_mae_all_class: 4.3413 - val_categorical_accuracy: 0.1417
Epoch 38/1000
134016/134466 [============================>.] - ETA: 0s - loss: 1.2772 - mae_all_class: 1.3937 - categorical_accuracy: 0.5457
Epoch 00038: val_cate

134208/134466 [============================>.] - ETA: 0s - loss: 1.2754 - mae_all_class: 1.3961 - categorical_accuracy: 0.5459
Epoch 00055: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.2755 - mae_all_class: 1.3985 - categorical_accuracy: 0.5458 - val_loss: 4.7446 - val_mae_all_class: 4.3358 - val_categorical_accuracy: 0.1420
Epoch 56/1000
134144/134466 [============================>.] - ETA: 0s - loss: 1.2703 - mae_all_class: 1.3836 - categorical_accuracy: 0.5487
Epoch 00056: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.2706 - mae_all_class: 1.3847 - categorical_accuracy: 0.5486 - val_loss: 4.7437 - val_mae_all_class: 4.3432 - val_categorical_accuracy: 0.1410
Epoch 57/1000
134208/134466 [============================>.] - ETA: 0s - loss: 1.2737 - mae_all_class: 1.3910 - categorical_accuracy: 0.5481
Epoch 00057: val_cate

134336/134466 [============================>.] - ETA: 0s - loss: 1.2750 - mae_all_class: 1.3909 - categorical_accuracy: 0.5471
Epoch 00074: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 106us/sample - loss: 1.2750 - mae_all_class: 1.3917 - categorical_accuracy: 0.5471 - val_loss: 4.7522 - val_mae_all_class: 4.3480 - val_categorical_accuracy: 0.1414
Epoch 75/1000
134272/134466 [============================>.] - ETA: 0s - loss: 1.2735 - mae_all_class: 1.3943 - categorical_accuracy: 0.5471
Epoch 00075: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.2738 - mae_all_class: 1.3989 - categorical_accuracy: 0.5471 - val_loss: 4.7610 - val_mae_all_class: 4.3341 - val_categorical_accuracy: 0.1417
Epoch 76/1000
134016/134466 [============================>.] - ETA: 0s - loss: 1.2752 - mae_all_class: 1.3933 - categorical_accuracy: 0.5476
Epoch 00076: val_cate

134144/134466 [============================>.] - ETA: 0s - loss: 1.2737 - mae_all_class: 1.3984 - categorical_accuracy: 0.5473
Epoch 00093: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 14s 105us/sample - loss: 1.2743 - mae_all_class: 1.4088 - categorical_accuracy: 0.5473 - val_loss: 4.7561 - val_mae_all_class: 4.3381 - val_categorical_accuracy: 0.1410
Epoch 94/1000
134400/134466 [============================>.] - ETA: 0s - loss: 1.2689 - mae_all_class: 1.3879 - categorical_accuracy: 0.5500
Epoch 00094: val_categorical_accuracy did not improve from 0.14363
134466/134466 [==============================] - 15s 110us/sample - loss: 1.2690 - mae_all_class: 1.3975 - categorical_accuracy: 0.5500 - val_loss: 4.7435 - val_mae_all_class: 4.3378 - val_categorical_accuracy: 0.1407
Epoch 95/1000
134080/134466 [============================>.] - ETA: 0s - loss: 1.2708 - mae_all_class: 1.3850 - categorical_accuracy: 0.5500
Epoch 00095: val_cate